In [1]:
import pandas as pd

In [2]:
tripadv_review = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\Cook Shop.csv', error_bad_lines=False);
tripadv_review

,r_points,r_text,place
0,50,Happy customer always.. excellent quality of f...,Cook Shop
1,50,The food is delicious!! There are various uniq...,Cook Shop
2,50,The food here is soooo delicious! Yhe waiter r...,Cook Shop
3,40,I had a business lunch there. Service was real...,Cook Shop
4,50,We had a great experience! Food was delicious....,Cook Shop
...,...,...,...
205,50,"Had a great dinner, excellent menu, not too ma...",Cook Shop
206,50,Excellent homemade bread!! We've had pork chop...,Cook Shop
207,40,"I visited this place three times so far, and p...",Cook Shop
208,50,I visited The Cookshop with friends who have l...,Cook Shop


In [3]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sent = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
sent.polarity_scores(tripadv_review.iloc[0]['r_text'])

{'neg': 0.0, 'neu': 0.553, 'pos': 0.447, 'compound': 0.8658}

In [6]:
tripadv_review['scores'] = tripadv_review['r_text'].apply(lambda review:sent.polarity_scores(review))
tripadv_review['compound'] = tripadv_review['scores'].apply(lambda d:d['compound'])
tripadv_review['score'] = tripadv_review['compound'].apply(lambda score: 'pos' if score >=0 else 'neg')
tripadv_review.head()

,r_points,r_text,place,scores,compound,score
0,50,Happy customer always.. excellent quality of f...,Cook Shop,"{'neg': 0.0, 'neu': 0.553, 'pos': 0.447, 'comp...",0.8658,pos
1,50,The food is delicious!! There are various uniq...,Cook Shop,"{'neg': 0.037, 'neu': 0.663, 'pos': 0.3, 'comp...",0.9574,pos
2,50,The food here is soooo delicious! Yhe waiter r...,Cook Shop,"{'neg': 0.0, 'neu': 0.6, 'pos': 0.4, 'compound...",0.9824,pos
3,40,I had a business lunch there. Service was real...,Cook Shop,"{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp...",0.9147,pos
4,50,We had a great experience! Food was delicious....,Cook Shop,"{'neg': 0.0, 'neu': 0.46, 'pos': 0.54, 'compou...",0.9694,pos


In [7]:
tripadv_review.to_csv(r'C:\Users\Dell\Desktop\sentiment_out7.csv')

In [8]:
pos_rev=tripadv_review[(tripadv_review['score'] .str.contains('pos'))]
pos_rev.to_csv(r'C:\Users\Dell\Desktop\sentiment_out7_pos.csv')
neg_rev=tripadv_review[(tripadv_review['score'] .str.contains('neg'))]
neg_rev.to_csv(r'C:\Users\Dell\Desktop\sentiment_out7_neg.csv')

In [9]:
pos_review_imp= pd.read_csv(r'C:\Users\Dell\Desktop\sentiment_out7_pos.csv')
neg_review_imp = pd.read_csv(r'C:\Users\Dell\Desktop\sentiment_out7_neg.csv')

In [10]:
from pycaret.nlp import *

In [11]:
exp_nlp = setup(data = pos_review_imp, target = 'r_text',
                custom_stopwords = [ 'hotel', 'room', 'nice', 'good', 'excellent','restaurant', 'food'])

Description,Value
session_id,2513
Documents,207
Vocab Size,1152
Custom Stopwords,True


In [12]:
#create an LDA model
lda_pos_custom = create_model('lda')
print(lda_pos_custom)

LdaModel(num_terms=1152, num_topics=4, decay=0.5, chunksize=100)


In [13]:
apply_lda_pos = assign_model(lda_pos_custom)
apply_lda_pos

,Unnamed: 0,r_points,r_text,place,scores,compound,score,Topic_0,Topic_1,Topic_2,Topic_3,Dominant_Topic,Perc_Dominant_Topic
0,0,50,happy customer always quality always fresh tas...,Cook Shop,"{'neg': 0.0, 'neu': 0.553, 'pos': 0.447, 'comp...",0.8658,pos,0.666039,0.009002,0.314752,0.010206,Topic 0,0.67
1,1,50,delicious various unique combination different...,Cook Shop,"{'neg': 0.037, 'neu': 0.663, 'pos': 0.3, 'comp...",0.9574,pos,0.418585,0.004235,0.493239,0.083941,Topic 2,0.49
2,2,50,recommend pull pork starter similar super tast...,Cook Shop,"{'neg': 0.0, 'neu': 0.6, 'pos': 0.4, 'compound...",0.9824,pos,0.003070,0.004233,0.426461,0.566236,Topic 3,0.57
3,3,40,really staff helpful steak cook perfectly frie...,Cook Shop,"{'neg': 0.0, 'neu': 0.757, 'pos': 0.243, 'comp...",0.9147,pos,0.706632,0.004577,0.283602,0.005188,Topic 0,0.71
4,4,50,great experience delicious creative cuisine ex...,Cook Shop,"{'neg': 0.0, 'neu': 0.46, 'pos': 0.54, 'compou...",0.9694,pos,0.006042,0.426468,0.558051,0.009439,Topic 2,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,205,50,great dinner menu many option cover wide range...,Cook Shop,"{'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'comp...",0.9194,pos,0.004175,0.005754,0.644128,0.345943,Topic 2,0.64
203,206,50,homemade service atmosphere outstanding,Cook Shop,"{'neg': 0.0, 'neu': 0.548, 'pos': 0.452, 'comp...",0.9243,pos,0.014959,0.020617,0.941050,0.023373,Topic 2,0.94
204,207,40,visit place time far plan warm friendly atmosp...,Cook Shop,"{'neg': 0.0, 'neu': 0.866, 'pos': 0.134, 'comp...",0.7823,pos,0.004175,0.005754,0.983548,0.006523,Topic 2,0.98
205,208,50,visit cookshop friend live long time familiar ...,Cook Shop,"{'neg': 0.0, 'neu': 0.744, 'pos': 0.256, 'comp...",0.9771,pos,0.002057,0.002835,0.417182,0.577927,Topic 3,0.58


In [14]:
evaluate_model(lda_pos_custom)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Frequency Plot', 'freque…

In [15]:
plot_model(lda_pos_custom, plot ='topic_model')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.215809  0.105586       1        1  62.764339
0      0.013104 -0.140175       2        1  16.532557
1     -0.120087 -0.100995       3        1  14.171245
3     -0.108826  0.135584       4        1   6.531860, topic_info=        Term        Freq       Total Category  logprob  loglift
86      menu   47.000000   47.000000  Default  30.0000  30.0000
229   dinner   25.000000   25.000000  Default  29.0000  29.0000
44      pork   14.000000   14.000000  Default  28.0000  28.0000
20     place  100.000000  100.000000  Default  27.0000  27.0000
1034    huge   14.000000   14.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
360     help    3.012585   11.854622   Topic4  -4.6219   1.3586
259    owner    3.415219   20.735815   Topic4  -4.4965   0.9249
806    local    2.575893   13.704744   Topic4  -4.7785   1.0569
426     hope    2.182297   12.659271   Topic4  -4.9443   0.9705
442   friend    3.006948   31.273621   Topic4  -4.6238   0.3866

[219 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
779       2  0.700735  accompany
10        1  0.718391       also
10        2  0.239464       also
10        3  0.059866       also
295       1  0.953928    amazing
...     ...       ...        ...
910       3  0.744289       wise
365       1  0.164384  wonderful
365       2  0.328769  wonderful
365       4  0.493153  wonderful
286       2  0.755465       work

[280 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 4])

In [16]:
tune_lda_positive = tune_model(model='lda', supervised_target=None, estimator='xgboost')

IntProgress(value=0, description='Processing: ', max=25)

Output()

Best Model: Latent Dirichlet Allocation | # Topics: 200 | Coherence: 0.4765


16 topics selected

## Negative reviews

In [17]:
exp_nlp_neg = setup(data = neg_review_imp, target = 'r_text',
                custom_stopwords = [ 'hotel', 'room', 'nice', 'good','restaurant', 'food','bad'])

Description,Value
session_id,3562
Documents,3
Vocab Size,41
Custom Stopwords,True


In [18]:
#create an LDA model
lda_neg_custom = create_model('lda')
print(lda_neg_custom)

LdaModel(num_terms=41, num_topics=4, decay=0.5, chunksize=100)


In [19]:
apply_lda_neg = assign_model(lda_neg_custom)
apply_lda_neg

,Unnamed: 0,r_points,r_text,place,scores,compound,score,Topic_0,Topic_1,Topic_2,Topic_3,Dominant_Topic,Perc_Dominant_Topic
0,62,10,visit couple day ago service terrible almost s...,Cook Shop,"{'neg': 0.204, 'neu': 0.737, 'pos': 0.059, 'co...",-0.7735,neg,0.001589,0.003852,0.990582,0.003976,Topic 2,0.99
1,94,50,tiny offer variety home make offardable price ...,Cook Shop,"{'neg': 0.099, 'neu': 0.819, 'pos': 0.082, 'co...",-0.1531,neg,0.002753,0.006674,0.007221,0.983352,Topic 3,0.98
2,168,10,maybe day awful ingredient fresh drink service...,Cook Shop,"{'neg': 0.22, 'neu': 0.709, 'pos': 0.071, 'com...",-0.7089,neg,0.003289,0.979861,0.008624,0.008226,Topic 1,0.98


In [20]:
evaluate_model(lda_neg_custom)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Frequency Plot', 'freque…

In [21]:
plot_model(lda_neg_custom, plot ='topic_model')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.030947  0.094019       1        1  48.779343
3      0.058133 -0.089763       2        1  27.827868
1     -0.092591 -0.008169       3        1  23.161856
0      0.003511  0.003913       4        1   0.230933, topic_info=          Term      Freq     Total Category  logprob  loglift
7          day  1.000000  1.000000  Default  30.0000  30.0000
17     service  1.000000  1.000000  Default  29.0000  29.0000
35       drink  0.000000  0.000000  Default  28.0000  28.0000
38  ingredient  0.000000  0.000000  Default  27.0000  27.0000
37       fresh  0.000000  0.000000  Default  26.0000  26.0000
..         ...       ...       ...      ...      ...      ...
5       couple  0.002424  1.098003   Topic4  -3.7127  -0.0450
20         way  0.002424  1.098056   Topic4  -3.7128  -0.0452
2   appreciate  0.002423  1.098116   Topic4  -3.7131  -0.0455
7          day  0.002425  1.589832   Topic4  -3.7122  -0.4146
17     service  0.002425  1.589783   Topic4  -3.7124  -0.4148

[183 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         1  0.910776            ago
1         1  0.910851         almost
2         1  0.910651     appreciate
33        3  1.087468          awful
3         1  0.910777       business
34        3  1.087465           come
4         1  0.910806     completely
5         1  0.910744         couple
6         1  0.910867      criticism
7         1  0.628997            day
7         3  0.628997            day
8         1  0.911024     definitely
21        2  1.034177         dinner
35        3  1.087469          drink
9         1  0.910748  establishment
36        3  1.087468           ever
10        1  0.910824       existent
11        1  0.910798       feedback
12        1  0.910823           form
37        3  1.087469          fresh
22        2  1.034132         friday
13        1  0.910846    furthermore
23        2  1.034072           home
38        3  1.087469     ingredient
24        2  1.034227          lunch
25        2  1.034216           make
39        3  1.087466          maybe
14        1  0.910877            non
26        2  1.034243     offardable
27        2  1.034219          offer
28        2  1.034091           open
29        2  1.034099          price
15        1  0.910848            run
16        1  0.910938            say
17        1  0.629017        service
17        3  0.629017        service
18        1  0.910742       terrible
30        2  1.034342          thing
40        3  1.087467          think
31        2  1.034299           tiny
32        2  1.034146        variety
19        1  0.910754          visit
20        1  0.910700            way, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])